In [1]:
'''
Demos written and developed by: Andrew E. Oriani 2020 EMAIL: oriani@uchicago.edu

'''
from PyInventor import *
import numpy as np
from numpy import pi, cos, sin, sqrt, log
import math
import glob
import time
import datetime

#Import plotting libraries for later
import matplotlib
from matplotlib import pyplot as plt

from matplotlib import cm
import matplotlib as mpl
from matplotlib.colors import BoundaryNorm 
from matplotlib.ticker import MaxNLocator

import pandas as pd

import pyEPR
from pyEPR import ansys as HFSS
from pyEPR.core_distributed_analysis import CalcObject

import scipy
from scipy import constants as const

from pint import UnitRegistry
ureg = UnitRegistry()
Q = ureg.Quantity
import os
import re as regex

# Field integrals code
import sys
sys.path.append('../python/src')
import field_integrals as fint

# Eigenmode:

<img src="Pictures/Cavity Eigenmode.png" alt="drawing" width="400">

Determines first 5 eigenmode frequencies in absence of any qubit. This demonstration also calculates S_e and S_m (electric and magnetic field participation of the cavity modes) for the first (TE_101) mode.

In [2]:
'''
HFSS eigenmode simulation Creation:

This sets up a standard eigenmode simulation without the qubit, just the bare cavity created above. It calculates
the mode frequencies, loss (with boundary impedances set) and the electric and magnetic surface participation 
ratios (S_e, S_m)

'''
# Project and HFSS design names for the bare waveguide model
dd = datetime.datetime.today().strftime('%y%m%d')
# dd = '230418'
solution_type = 'eigenmode'

# solution_type = 'driven_modal'
overwrite = False
ref_name = 'coax_cavity_underneath_substrate_230131'
ref_name = 'two_post_coupler_eigenmode'
# ref_name = 'planar_pads_l_coupled_three_cavities_degen'
project_name=f'{ref_name}'
design_name=f'coupler_only_jj'


#use file location path:
HFSS_path=os.getcwd()

full_path=HFSS_path+'\\HFSS\\'+project_name+'.aedt'

HFSS_app=HFSS.HfssApp()
HFSS_desktop=HFSS_app.get_app_desktop()

project=HFSS_desktop.open_project(full_path)

if project==None:
    project=HFSS_desktop.new_project()
    project.save(full_path)
    
project.save(full_path)
project.make_active()
    
if design_name in project.get_design_names():
    if overwrite==True:
        project.delete_design(design_name)
        project.save()
        
        # Setup a driven modal design type
        if solution_type == 'driven_modal':
            EM_design=project.new_dm_design(design_name)
        elif solution_type == 'eigenmode':
            EM_design=project.new_em_design(design_name)
        else:
            raise ValueError(f'Solution type ({solution_type}) not recognized.')
    else:
        EM_design=project.get_design(design_name)
        
else:
    if solution_type == 'driven_modal':
        EM_design=project.new_dm_design(design_name)
    elif solution_type == 'eigenmode':
        EM_design=project.new_em_design(design_name)
    else:
        raise ValueError(f'Solution type ({solution_type}) not recognized.')
        
EM_design.make_active()
model=HFSS.HfssModeler(EM_design)

project=HFSS_desktop.open_project(full_path)

if project==None:
    project=HFSS_desktop.new_project()
    project.save(full_path)
    
project.make_active()
    
if design_name in project.get_design_names():
    if overwrite==True:
        project.delete_design(design_name)
        project.save()
        EM_design=project.new_em_design(design_name)
    else:
        EM_design=project.get_design(design_name)
        
else:
    EM_design=project.new_em_design(design_name)
        
EM_design.make_active()
model=HFSS.HfssModeler(EM_design)

project_name: two_post_coupler_eigenmode
path: C:\Users\materise1\tcav\ipynbs\HFSS\two_post_coupler_eigenmode.aedt
project_name: two_post_coupler_eigenmode
path: C:\Users\materise1\tcav\ipynbs\HFSS\two_post_coupler_eigenmode.aedt


### Eigenmode simulation and $L_J$ sweep

In [ ]:
#setup the analysis and optimetrics
opti_setup = HFSS.Optimetrics(EM_design)

#set the optimetrics step size and number of steps
step = .2
num_step = 1

sweep_type = 'decade'

#create optimetrics setup
# swp_var = L_jj
# swp_var_unit = 'nH'
# swp_var = xoffset
# swp_var_unit = 'mm'
start = 0.1e-9
stop = 100e-9
dstep = 3
num_step = int(np.round((stop-start)/dstep) + 1)
num_step = 3

tstamp = datetime.datetime.today().strftime('%y%m%d_%H_%M_%S')
if sweep_type == 'decade':
    opti_setup.create_setup(variable=swp_var,
                        swp_type= 'decade',
                        swp_params=('%.2f%s'%(start, swp_var_unit),
                                    '%.2f%s'%(stop, swp_var_unit), 
                                    str(num_step)),
                        name=f'{swp_var}_Sweep_{tstamp}',
                        save_fields=True,
                        solve_with_copied_mesh_only=False,
                        copy_mesh=False)
elif sweep_type == 'linear':
    opti_setup.create_setup(variable=swp_var,
                        swp_type= 'linear_step',
                        swp_params=('%.2f%s'%(start, swp_var_unit),
                                    '%.2f%s'%(stop, swp_var_unit), 
                                    '%.2f%s' \
                                    % (((stop - start) / num_step),
                                       swp_var_unit)),
                        name=f'{swp_var}_Sweep_{tstamp}',
                        save_fields=True,
                        solve_with_copied_mesh_only=False,
                        copy_mesh=False)
else:
    raise ValueError(f'sweep_type ({sweep_type}) unrecognized.')
            
#solve it
opti_setup.solve_setup(f'{swp_var}_Sweep_{tstamp}')

#save project
project.save()

In [ ]:
def set_axes_fonts(ax, fsize):
    """
    Set axes font sizes because it should be abstracted away
    """
    for tick in ax.get_xticklabels():
        tick.set_fontsize(fsize)
    for tick in ax.get_yticklabels():
        tick.set_fontsize(fsize)

def set_xaxis_rot(ax, angle=45):
    """
    Rotate the x-axis labels
    """
        
    for tick in ax.get_xticklabels():
        tick.set_rotation(angle)

def set_leg_outside(ax, fsize):
    """
    Sets the legend location outside
    """
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    # Put a legend to the right of the current axis
    hdls, legs = ax.get_legend_handles_labels()
    leg = ax.legend(hdls, legs, loc='center left', bbox_to_anchor=(1, 0.5), \
                    fontsize=fsize, framealpha=0.)

    return leg

In [ ]:
def plot_2d_cmap(x, y, z, fname,                                                
                 xstr='', ystr='',                                              
                 tstr='', cbar_str='',                                          
                 cmap=cm.inferno,
                 xyscales={'x' : 'linear', 'y': 'linear'},
                 interp_zeros=False,
                interp_std_dev=10,
                image_rot_angle=0,
                norm_type='linear',
                zref=None, 
                fsize=30,
                tsize=36):
    """                                                                         
    Plot 2D colormap data such that                                             
                                                                                
         -----------------------                                                
         |                     |                                                
         |                     |                                                
    y    |          z          |                                                
         |                     |                                                
         |                     |                                                
         -----------------------                                                
                                                                                
                    x                                                           
                                                                                
    Parameters:                                                                 
    ----------                                                                  
                                                                                
    x, y:       independent variables                                           
    z:          resulting data, z = z(x, y)                                     
    fname:      output figure filename, relative path with file extension       
    xstr:       x-axis label                                                    
    ystr:       y-axis label                                                    
    tstr:       title label                                                     
    cbar_str:   lable for color bar                                             
    cmap:       colormap to use on the 2D figure output                         
                                                                                
    """
    # Interpolate the data
    if interp_zeros:
        hot_pixels, z = fix_outlier_pixels(z, tolerance=interp_std_dev)
    
    # Setup the color map, normalizations, etc
    if norm_type == 'linear':
        norm = mpl.colors.Normalize(z.min(), z.max())
    elif norm_type == 'log':
        if zref:
            norm = mpl.colors.LogNorm(min(zref), max(zref))
        else:
            norm = mpl.colors.Normalize(min(zref), max(zref))    
    elif (norm_type == 'ref') and zref:
        norm = mpl.colors.Normalize(min(zref), max(zref))
    else:
        raise ValueError(f'norm_type ({norm_type}) not recognized')                                         
                                                                                
    # Setup the figure and axes                                                 
    fig, ax = plt.subplots(1, 1, figsize=(10, 8), tight_layout=True)
    
    # Set the scale bar and levels
    if (norm_type == 'ref') and zref:
        levels = MaxNLocator(nbins=20).tick_values(min(zref), max(zref))
    else:
        levels = MaxNLocator(nbins=20).tick_values(z.min(), z.max())
    if image_rot_angle:
        z = scipy.ndimage.rotate(z, image_rot_angle, reshape=True)
    #plt1 = ax.imshow(z, cmap=cmap, norm=norm)
    plt1 = ax.pcolormesh(x, y, z, cmap=cmap, norm=norm)
    ax.set_xlabel(xstr, fontsize=fsize)                                         
    ax.set_ylabel(ystr, fontsize=fsize)                                         
    ax.set_title(tstr, fontsize=tsize)
    
    # Rotate the xlabels. Set both x and y labels to headers[1:]
    ax.set_xscale(xyscales['x'])
    ax.set_yscale(xyscales['y'])
    # set_xaxis_rot(ax, 45)
                                                                                
    # Set the axis tick labels to a reasonable size                             
    set_axes_fonts(ax, fsize)                                                   
                                                                                
    # Set the color bar, offset the title slightly from top                     
    cbar = fig.colorbar(plt1, ax=ax)                                            
    cbar.ax.set_title(cbar_str, fontsize=fsize, y=1.025)                        
    cbar.ax.tick_params(labelsize=fsize)                                        
                                                                                
    # Write the results to file
    print(f'Writing {fname} ...')
    fig.savefig(fname, format='pdf', transparent=True)                  

## Read the electric field data to perform the integrals

In [ ]:
# Setup the pyEPR project
Nmodes = 2
pinfo = pyEPR.ProjectInfo(HFSS_path+'\\HFSS',  
                         project_name,
                         design_name)
# Setup for EPR anaylsis
eprh = pyEPR.DistributedAnalysis(pinfo)

# Read off the inductances and x-offsets from the variations
variations = eprh.solutions.list_variations()
Ljstrs = []; Lj_vals = []
#xoffstrs = []; xoff_vals = []
for v in variations:
    Ljstrs.append(regex.findall('LJ=\'(.*?nH)', v)[0].replace('.', 'p'))
    Lj_vals.append(float(regex.findall('LJ=\'(.*?)nH', v)[0]))
 #   xoffstrs.append(regex.findall('x_offset=\'(.*?mm)', v)[0].replace('.', 'p'))
 #   xoff_vals.append(regex.findall('x_offset=\'(.*?mm)', v)[0])

In [ ]:
Lj_sort_idxs = np.argsort(Lj_vals)
Lj_vals = np.unique(np.asarray(Lj_vals)[Lj_sort_idxs])
Ljstrs = np.asarray(Ljstrs)[Lj_sort_idxs]

In [ ]:
Lj_sort_idxs

In [ ]:
Lj_vals

In [ ]:
Ljstrs

In [ ]:
vstrs = [f'LJ{l}' for l in Ljstrs]

In [ ]:
vstrs

In [ ]:
vars_sorted = np.asarray(eprh.variations)[Lj_sort_idxs]

In [ ]:
[eprh._get_lv(v) for v in vars_sorted]

## Need to manually create Vector_E plot in GUI first
* TODO: Create a new field plot, then export the field data with ansys.py

In [ ]:
# Build a string for each variation to label the output files
Nmodes = 2
for idx, v in enumerate(vars_sorted):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Variation: {vstrs[idx]}')
    for kp in range(Nmodes):
        # Set the solution to mode kp
        eprh.solutions.set_mode(kp+1)
        # time.sleep(0.5)
        print(f'Mode-{kp+1}')
        des = EM_design._design
        omod = des.GetModule('FieldsReporter')
        omod.ExportFieldPlot("Vector_E1", False, f"Z:\\tcav\\data\\efields_two_post_cpl_only_mode_{kp+1}_{vstrs[idx]}.aedtplt")
print('Finished writing field data to file.')

## Need to manually create Vector_H (90 degrees phase) plot in GUI first

In [ ]:
# Build a string for each variation to label the output files
Nmodes = 2
for idx, v in enumerate(vars_sorted):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Variation: {vstrs[idx]}')
    for kp in range(Nmodes):
        # Set the solution to mode kp
        eprh.solutions.set_mode(kp+1)
        # time.sleep(0.5)
        print(f'Mode-{kp+1}')
        des = EM_design._design
        omod = des.GetModule('FieldsReporter')
        omod.ExportFieldPlot("Vector_H1", False, f"Z:\\tcav\\data\\hfields_two_post_cpl_only_mode_{kp+1}_{vstrs[idx]}.aedtplt")
print('Finished writing field data to file.')

## Need to manually create Vector_Jsurf1 (90 degrees phase) plot in GUI first

In [ ]:
# Build a string for each variation to label the output files
Nmodes = 2
for idx, v in enumerate(vars_sorted):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Variation: {vstrs[idx]}')
    for kp in range(Nmodes):
        # Set the solution to mode kp
        eprh.solutions.set_mode(kp+1)
        # time.sleep(0.5)
        print(f'Mode-{kp+1}')
        des = EM_design._design
        omod = des.GetModule('FieldsReporter')
        omod.ExportFieldPlot("Vector_Jsurf1", False, f"Z:\\tcav\\data\\jfields_two_post_cpl_only_mode_{kp+1}_{vstrs[idx]}.aedtplt")
print('Finished writing field data to file.')

## Get the currents from pyEPR

In [ ]:
Ikkp = np.zeros([len(vars_sorted), len(vars_sorted), Nmodes, Nmodes])
jj_area = (50e-6)**2
for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx, v in enumerate(vars_sorted):
            for idx0, v0 in enumerate(vars_sorted):
                # Set the solution to mode k
                eprh.solutions.set_mode(k+1)
                calc = CalcObject([], eprh.setup)
                calc = calc.getQty('Jsurf').mag().integrate_surf(name ='jj')
                
                # Set the variation to v
                eprh.set_variation(v)
                lv = eprh._get_lv(v)
                Ik = calc.evaluate(lv=lv, phase=90.) / jj_area
                
                # Set the solution to mode kp
                eprh.solutions.set_mode(kp+1)
                calc = CalcObject([], eprh.setup)
                calc = calc.getQty('Jsurf').mag().integrate_surf(name ='jj')
                
                # Set the variation to v0
                eprh.set_variation(v0)
                lv0 = eprh._get_lv(v0)
                Ikp = calc.evaluate(lv=lv0, phase=90.) / jj_area
            
                print(f'(k, kp): ({k+1}, {kp+1}), (LJ, LJ0): ({v}, {v0})')
                Ikkp[idx, idx0, k, kp] = Ik * Ikp 

In [ ]:
Ikkp[:, :, 0, 0].shape

In [ ]:
from importlib import reload
reload(fint)

In [ ]:
# Single point of entry inputs to class
Nmodes = 2
modes = np.linspace(1, Nmodes, Nmodes, dtype=int)
print(f'modes: {modes}')
# LJs = np.asarray([0.1, 1., 10., 100.])
# LJstrs = ['0p1', '1', '10']

# Instantiate the class object
data_path = f'../data'
myfields = fint.FieldIntegrals(Lj_vals, Ljstrs,
                               modes, data_path, debug=True,
                               has_updated_fields=True,
                               has_normalized_fields=False,
                               normalize_fields=True,
                               drop_max_field=1,
                               filename_format = f'jfields_two_post_cpl_only_mode_%d_LJ%s.aedtplt',
                               timeout_limit=30.)
abscissa, efields = myfields.load_fields(return_fields=True)
keys = list(efields.keys())

Ckkp = myfields.overlap_matrix_points(fname=None, cidxs=[0, 1],
                                      drop_points=1,#'../data/Akkp_nondegenerate_flutes_{dd}.csv',
                                      return_matrix=True)

In [ ]:
# Single point of entry inputs to class
Nmodes = 2
modes = np.linspace(1, Nmodes, Nmodes, dtype=int)
print(f'modes: {modes}')
# LJs = np.asarray([0.1, 1., 10., 100.])
# LJstrs = ['0p1', '1', '10']

# Instantiate the class object
data_path = f'../data'
myfields = fint.FieldIntegrals(Lj_vals, Ljstrs,
                               modes, data_path, debug=True,
                               has_updated_fields=True,
                               has_normalized_fields=False,
                               normalize_fields=True,
                               drop_max_field=1,
                               filename_format = f'hfields_two_post_cpl_only_mode_%d_LJ%s.aedtplt',
                               timeout_limit=30.)
abscissa, efields = myfields.load_fields(return_fields=True)
keys = list(efields.keys())

Bkkp = myfields.overlap_matrix_points(fname=None, cidxs=[0, 1],
                                      drop_points=1,#'../data/Akkp_nondegenerate_flutes_{dd}.csv',
                                      return_matrix=True)

In [ ]:
Ckkp

In [ ]:
a = np.reshape([1,2], [1, 2]); b = np.reshape([3, 4], [1, 2]); c = np.reshape([5, 6], [1, 2])

In [ ]:
np.hstack((np.vstack((a, b, c)), np.ones((3, 1))))

In [ ]:
Bkkp

## Call the field integral code

In [ ]:
# Single point of entry inputs to class
Nmodes = 2
modes = np.linspace(1, Nmodes, Nmodes, dtype=int)
print(f'modes: {modes}')
# LJs = np.asarray([0.1, 1., 10., 100.])
# LJstrs = ['0p1', '1', '10']

# Instantiate the class object
data_path = f'../data'
myfields = fint.FieldIntegrals(Lj_vals, Ljstrs,
                               modes, data_path, debug=True,
                               has_updated_fields=True,
                               has_normalized_fields=False,
                               normalize_fields=True,
                               drop_max_field=1,
                               filename_format = f'efields_two_post_cpl_only_mode_%d_LJ%s.aedtplt',
                               timeout_limit=30.)
abscissa, efields = myfields.load_fields(return_fields=True)
keys = list(efields.keys())

Akkp = myfields.overlap_matrix_points(fname=None, cidxs=[0, 1],
                                      drop_points=1,#'../data/Akkp_nondegenerate_flutes_{dd}.csv',
                                      return_matrix=True)

In [ ]:
len(Akkp)

In [ ]:
Akkpcopy = Akkp.copy()

## Repair the $A_{kk'}$ and $B_{kk'}$ matrices

In [ ]:
A34 = np.zeros([len(Lj_vals), len(Lj_vals)])
A43 = np.zeros([len(Lj_vals), len(Lj_vals)])

for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx in range(len(Lj_vals)):
            for idx0 in range(len(Lj_vals)):
                if k != kp:
                    key34 = f'kkp{k}_{kp}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    key43 = f'kkp{kp}_{k}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    A34[idx, idx0] = Akkpcopy[key34]
                    A43[idx, idx0] = Akkpcopy[key43]
                    Akkp[key34] = (A34[idx, idx0] - A43[idx, idx0]) / 2
                    Akkp[key43] = (A43[idx, idx0] - A34[idx, idx0]) / 2

In [ ]:
Akkpcopy[key43]

In [ ]:
Akkpcopy['kkp3_4_LJLJp_100nH_100nH']

In [ ]:
ww, qq = eprh.solutions.eigenmodes()

In [ ]:
ww

In [ ]:
qq

## Get the eigenmode frequencies and losses
* Note that frequencies are in GHz here

In [ ]:
wk = {}; Qk = {};
vstrs = [f'LJ{l}' for l in Ljstrs]
for idx, v in enumerate(eprh.variations):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Eigenmodes for variation {vstrs[idx]} ...')
    ww, qq = eprh.solutions.eigenmodes(lv=lv)
    key = f'{vstrs[idx]}'
    wk.update({key : ww})
    Qk.update({key : [ww1/qq1 for ww1, qq1 in zip(ww, qq)]})

In [ ]:
wk

In [ ]:
Qk

In [ ]:
LJ_dict = dict({k : v for k, v in zip(Ljstrs, Lj_vals)})

In [ ]:
LJ_dict

## Plot the modes vs. inductances

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
fsize = 20
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
idx_start = 4
Nmodes = 2
for k in range(Nmodes):
    wppp = np.array([wk[f'LJ{key}'][k] for key in Ljstrs])
    ax.plot(Lj_vals, wppp,
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
    #ax.plot(LJs/1e-9, [wp0[idx]/(2*np.pi*fscale)] * len(LJs),
    #        '--', color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
#ax.set_ylim([5.88, 5.95])
set_axes_fonts(ax, fsize)
leg = set_leg_outside(ax, lsize)
ax.set_xscale('log')
fig.savefig(f'../figs/modes_bare_remesh2_nondegen_{dstr}.pdf', 
            format='pdf', bbox_extra_artists=(leg, ), bbox_inches='tight',
           transparent=True)

## Q of the modes

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale = 1e9
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
T1trans = 100
for k in range(Nmodes):
    Q = np.array([Qk[f'LJ{key}'][k] for key in Ljstrs])
    wppp = 2*np.pi*np.array([wk[f'LJ{key}'][k] for key in Ljstrs])*fscale
    T1tot = T1nom * T1trans / (T1nom + T1trans)
    ax.plot(Lj_vals, Q,
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$Q$', fontsize=fsize)
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
#ax.set_ylim([1e-2, 350])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/T1_modes_coupled_remesh2_eigenmode_{dstr}.pdf',
            format='pdf')

# Plot the eigenmode $T_1$ values

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale = 1e9
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
T1trans = 100
for k in range(Nmodes):
    Q = np.array([Qk[f'LJ{key}'][k] for key in Ljstrs])
    wppp = 2*np.pi*np.array([wk[f'LJ{key}'][k] for key in Ljstrs])*fscale
    T1nom = 1e6 * Q / wppp
    T1tot = T1nom * T1trans / (T1nom + T1trans)
    ax.plot(Lj_vals, T1nom,
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$T_1$ [$\mu$s]', fontsize=fsize)
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
#ax.set_ylim([1e-2, 350])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/T1_modes_coupled_remesh2_eigenmode_{dstr}.pdf',
            format='pdf')

## Compute the $g_{kk'}$ and $\widetilde{\omega}_{k}$ from $A_{kk'}$

In [ ]:
"""
Compute the couplings from Akk, wk values
"""
# Iterate over all modes and inductances
gkk = {}
idx_start = 2; idx_stop = -1
Nmodes = 4
for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx, LJ in enumerate(Lj_vals):
            
            # Outside sqrt(wk wk') factor
            wkey = f'LJ{Ljstrs[idx]}'
            wkwkp = np.sqrt(wk[wkey][k] * wk[wkey][kp])
            
            for idx0, LJ0 in enumerate(Lj_vals):
                
                # Key to index gkk'
                gkey = f'kkp{k+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                Akkpsum = 0
                print(f'Computing g at {gkey}, wkwkp: {wkwkp:.2e} ...')
                for kpp in range(Nmodes):
                    key = f'kkp{kpp+1}_{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    keyp = f'kkp{kpp+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    Akppk = Akkp[key]
                    Akppkp = Akkp[keyp]
                    Akkpsum += Akppk * Akppkp

                # Store the results
                if (k != kp):
                    gkk.update({gkey : wkwkp * Akkpsum})

In [ ]:
'kkp1_1_LJLJp_0p10000000000000001nH_0p10000000000000001nH' in gkk.keys()

### Collect the results as a matrix

In [ ]:
gkk_matrix = np.zeros([Nmodes, Nmodes, len(Lj_vals), len(Lj_vals)])
for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx, LJ in enumerate(Lj_vals):
            for idx0, LJ0 in enumerate(Lj_vals):
                if k != kp:
                    gkey = f'kkp{k+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    gkk_matrix[k, kp, idx, idx0] = gkk[gkey]

In [ ]:
gkk_matrix

In [ ]:
"""
Compute the couplings from Akk, wk values
"""
# Iterate over all modes and inductances
wkt = {}
idx_start = 2; idx_stop = -1
Nmodes = 4
for k in range(Nmodes):
    for idx, LJ in enumerate(Lj_vals):

        # Outside wk factor
        wkey = f'LJ{Ljstrs[idx]}'
        wkp = wk[wkey][k]

        for idx0, LJ0 in enumerate(Lj_vals):

            # Key to index gkk'
            wwkey = f'kkp{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
            Akkpsum = 0
            # print(f'Computing wkp at {wwkey}, wkwkp: {wkp:.2e} ...')
            for kpp in range(Nmodes):
                key = f'kkp{kpp+1}_{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                Akppk = Akkp[key]
                Akkpsum += Akppk**2

            # Store the results
            wkt.update({wwkey : wkp * Akkpsum})

In [ ]:
len(wkt)

In [ ]:
wkt_matrix = np.zeros([Nmodes, len(Lj_vals), len(Lj_vals)])

## Compute the $\widetilde{\omega}_k(L_J, L_{J_0})$ matrix

In [ ]:
for k in range(Nmodes):
    for idx, LJ in enumerate(Lj_vals):
        for idx0, LJ0 in enumerate(Lj_vals):
            wwkkey = f'kkp{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
            wkt_matrix[k, idx, idx0] = wkt[wwkkey]

### Sort the frequencies for each $(L_J, L_{J_0})$ pair

In [ ]:
wkt_matrix_sorted = np.sort(wkt_matrix, axis=0)
wkt_sort_idx = np.argsort(wkt_matrix, axis=0)

gkk_sort_idx = wkt_sort_idx[np.newaxis, :, :, :]

In [ ]:
gkk_sort_idx.shape

In [ ]:
gkk_matrix_sorted = gkk_matrix[gkk_sort_idx]

In [ ]:
gkk_matrix_sorted.shape

## Plot the sorted $\widetilde{\omega}_k(L_J, L_{J_0})$ values

In [ ]:
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)

for idx_fixed in range(len(Lj_vals)):
    print(f'LJ0-{idx_fixed} ...')
    fig, ax = plt.subplots(1, 1, tight_layout=True)
    for k in range(Nmodes):
        ax.plot(Lj_vals, wkt_matrix_sorted[k, :, idx_fixed],
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
    ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
    ax.set_ylabel(r'$\widetilde{\omega}/2\pi$ [GHz]', fontsize=fsize)
    set_xaxis_rot(ax, 45)
    set_axes_fonts(ax, fsize)
    leg = set_leg_outside(ax, lsize)
    ax.set_xscale('log')
    fig.savefig(f'../figs/modes_eigenmode_remesh2_modified_LJ0_{Ljstrs[idx_fixed]}_{dstr}.pdf', 
                format='pdf', bbox_extra_artists=(leg, ), bbox_inches='tight',
               transparent=True)

In [ ]:
'kkp1_LJLJp_0p10000000000000001nH_56p234132519034901nH' in wkt.keys()

## Plot $g_{k_1,k_2}$ vs. $L_J, L_{J_0}$

In [ ]:
gmax = np.max(np.array(list(gkk.values()))) * 1e3
gmin = np.min(np.array(list(gkk.values()))) * 1e3

In [ ]:
gmax

In [ ]:
gmin

In [ ]:
#plot_2d_cmap(ydata/ymax,
#    zdata/zmax,
for k1 in [1,2,3,4]:
    for k2 in [1, 2, 3, 4]:
        if k1 != k2:
            gkk12 = np.array([[gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                               for idx in range(len(Lj_vals))] \
                               for idx0 in range(len(Lj_vals))])*1e3
            plot_2d_cmap(# LJs/1e-9,
                #Lj_vals[idx_start:idx_stop],
                #Lj_vals[idx_start:idx_stop],
                Lj_vals,
                Lj_vals,
                gkk12,
                # gkk_matrix_sorted[k1, k2, :, :],
                f'../figs/g{k1}_{k2}_remesh2_sorted_vs_LJLJ0_{dd}.pdf',
                xstr=r'$L_{J_0}$ [nH]',
                ystr=r'$L_{J}$ [nH]',
                tstr='',
                xyscales={'x' : 'log', 'y' : 'log'},
                cbar_str=r'$g_{%d%d}$ [MHz]' % (k1, k2),
                interp_zeros=False,
                interp_std_dev=3,
                image_rot_angle=0,
                norm_type='ref',
                zref=[gmin, gmax],
                fsize=40,
                tsize=46)

## Plot $B_{k_1,k_2}(L_{J}, L_{J_0})$ vs. $L_J, L_{J_0}$

In [ ]:
# Amax = np.max(list(Akkp.values()))
# Amin = np.min(list(Akkp.values()))

Bmax = np.max(np.abs(list(Bkkp.values())))
Bmin = np.min(np.abs(list(Bkkp.values())))

In [ ]:
Ljstrs

In [ ]:
Bmin, Bmax

In [ ]:
Amax

In [ ]:
Amin

In [ ]:
for k1 in range(1, Nmodes+1):
    for k2 in range(1, Nmodes+1):

        Akk12 = np.array([[Bkkp[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                           for idx in range(len(Lj_vals))] \
                           for idx0 in range(len(Lj_vals))])
        
        
        plot_2d_cmap(
            Lj_vals,
            Lj_vals,
            np.abs(Akk12),
            f'../figs/B{k1}_{k2}_two_post_coupler_only_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            cmap=cm.viridis,
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$|B_{%d%d}|$' % (k1, k2),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='log',
            zref=[Bmin, Bmax],
            fsize=40,
            tsize=46)

In [ ]:
Amax = np.max(np.abs(list(Akkp.values())))
Amin = np.min(np.abs(list(Akkp.values())))
for k1 in range(1, Nmodes+1):
    for k2 in range(1, Nmodes+1):

        Akk12 = np.array([[Akkp[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                           for idx in range(len(Lj_vals))] \
                           for idx0 in range(len(Lj_vals))])
        
        
        plot_2d_cmap(
            Lj_vals,
            Lj_vals,
            np.abs(Akk12),
            f'../figs/A{k1}_{k2}_two_post_coupler_only_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            cmap=cm.viridis,
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$|A_{%d%d}|$' % (k1, k2),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='log',
            zref=[Amin, Amax],
            fsize=40,
            tsize=46)

In [ ]:
Imax = np.max(np.abs(Ikkp))
Imin = np.min(np.abs(Ikkp))
for k1 in range(Nmodes):
    for k2 in range(Nmodes):

        Akk12 = Ikkp[:, :, k1, k2]
        
        plot_2d_cmap(
            Lj_vals,
            Lj_vals,
            np.abs(Akk12),
            f'../figs/I{k1}_{k2}_two_post_coupler_only_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            cmap=cm.viridis,
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$|I_{%d}I_{%d}|$' % (k1+1, k2+1),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='log',
            zref=[Imin, Imax],
            fsize=40,
            tsize=46)
        

In [ ]:
wmax = np.max(np.array(list(wkt.values())))
wmin = np.min(np.array(list(wkt.values())))

wmax

wmin

#plot_2d_cmap(ydata/ymax,
#    zdata/zmax,
# for k1 in [1,2,3,4]:
for k1 in range(Nmodes):
        # wkk12 = np.array([[wkt[f'kkp{k1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
        #                   for idx in range(len(Lj_vals))] \
        #                   for idx0 in range(len(Lj_vals))])
        plot_2d_cmap(# LJs/1e-9,
            Lj_vals,
            Lj_vals,
            #wkk12,
            wkt_matrix_sorted[k1, :, :],
            # f'../figs/w_{k1}_vs_LJLJ0_{dd}.pdf',
            f'../figs/w_{k1+1}_remesh2_sorted_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$\omega_{%d}$ [GHz]' % (k1),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='ref',
            zref=[wmin, wmax],
            fsize=40,
            tsize=46)

## Compute the derivatives to estimate the driven coupling strengths

dLJ = 1 # 10 nH as a possible driving point
for k1 in [1,2,3,4]:
    for k2 in [1, 2, 3, 4]:
        if k1 != k2:
            gkk12 = np.array([[gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                               for idx in range(len(Lj_vals))] \
                               for idx0 in range(len(Lj_vals))])
            gkk12_grad = np.array([np.gradient(gkk12[:, j], Lj_vals) for j in range(len(Lj_vals))])
            epskk = gkk12_grad * dLJ * 1e3
            # Write the results to file
            plot_2d_cmap(# LJs/1e-9,
            Lj_vals,
            Lj_vals,
            epskk,
            f'../figs/eps_{k1}_{k2}_remesh2_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$\epsilon_{g_{%d %d}}$ [MHz]' % (k1, k2),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='ref',
            zref=[gmin, gmax],
            fsize=40,
            tsize=46)

## Plot $g_{12}$ for a fixed $L_{J_0}$

LJidx0 = 0
dLJ = 10 # 10 nH as a possible driving point
k1 = 1; k2 = 2

fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
fsize = 20
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
idx_start = 4
cidx = 0
for idx0 in [8, 10, 12, 14]:
    gkk12 = np.array([gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[LJidx0]}'] \
                               for idx in range(len(Lj_vals))])
    gkk12_grad = np.gradient(gkk12, Lj_vals)
    epskk = gkk12_grad * Lj_vals[idx0] * 1e3
    ax.plot(Lj_vals, epskk / 2,
        'o-', color=cycle[cidx%clen],
           label=r'$\delta L_{J}$ = %.2f nH' % Lj_vals[idx0])
    cidx += 1

ax.axhline(y=0, color='k', ls='--')
ax.set_title(r'$L_{J_0} = %.2f$ nH' % Lj_vals[LJidx0],
             fontsize=fsize)
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$\frac{1}{2}\epsilon_{g_{%d%d}}$ [MHz]' % (k1, k2), fontsize=fsize)
set_xaxis_rot(ax, 45)
#ax.set_ylim([-0.5, 1.5])
set_axes_fonts(ax, fsize)
ax.set_xscale('log')
leg = set_leg_outside(ax, lsize)
fig.savefig(f'../figs/epsg_{k1}{k2}_bbq_1_10_100nH_remesh2_{dstr}.pdf', 
            bbox_extra_artists=(leg, ), bbox_inches='tight',
            format='pdf', transparent=True)

### Get the interpolating sweep data to estimate resonance positions

In [ ]:
fsweep_interp = HFSS.HfssFrequencySweep(DM_setup, 'Interpolating_0')

report_im = fsweep_interp.create_report('Interpolating_0', expr='im(Y(1,1))')
yim = report_im.get_arrays()

# Identify the zero crossings as resonances
zero_xing_idxs = np.where(np.diff(np.sign(yim[1,:])) > 0)[0]
resonances = yim[0,zero_xing_idxs]

print(f'resonances: {resonances}')

In [ ]:
# Plot the results, save the figure(s)
fsize = 20
fscale = 1e9
#wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gD, chippp = \
#get_wp_Cp_Lp(fdata, ydata, LJ0=None, CJ0=None, fscale=1e9, use_Zq=False)

fig, ax = plt.subplots(1, 1, tight_layout=True)
ax.plot(yim[0,:], yim[1,:], '-', label='Zero Crossings')
ax.plot(resonances,
        yim[1,zero_xing_idxs],
        'o', label='Zero Crossings')
# ax.set_xlim([5.8, 6.1])
ax.set_ylim([-0.1, 0.1])
ax.axhline(y=0, ls='--', color='k')
# ax.plot(fdata, ydata.imag)
set_xaxis_rot(ax, 45)
ax.set_ylabel(r'Im $Y(\omega)$', fontsize=fsize)
ax.set_xlabel(r'Frequency [GHz]', fontsize=fsize)
set_axes_fonts(ax, fsize)

In [ ]:
resonances

#### Bypass the interpolating sweep if frequencies already known

In [ ]:
# resonances = np.array([ 6.0928,  6.1684,  6.1692,  6.2692,  7.3356,  7.3816,  7.4712,
#        7.506 ,  9.3064,  9.3292,  9.588 ,  9.6076, 11.5804, 11.6684,
#       11.7568, 11.95  , 12.604 , 12.6944, 12.8012, 12.8048])
# resonances = np.array([ 6.0928,  6.1692,  6.2692,  7.3816,  7.4712,
#        7.506 , 9.3292,  9.588 ,  9.6076, 11.5804, 
#       11.7568, 11.95, 12.6944, 12.8048])

# Use half the minimum spacing between resonances as a the bandwidth
Nmodes = 10
# resonances = resonances[0:Nmodes]
df = 0.8 * np.diff(np.hstack((f1, resonances, f2))).min()

In [ ]:
df

### Discrete sweep to resolve resonances

In [ ]:
if solution_type == 'driven_modal':
    # Setup the driven-modal simulation
    if not DM_setup:
        DM_setup = EM_design.create_dm_setup()
        
    # Discrete count
    count2 = 201
    count1 = 51
    
    ## This should resolve the issue of overlapping sweeps
    ## We advance the left boundary by a small amount away
    ## from the previous right boundary
    dff = df / count
    
    # Identify resonances from interpolating sweep manually (for now)
    ## Start at the first sweep frequency
    f1r = f1
    
    # df  = 0.01 # 10 MHz window
    # Iteratve over all resonances and shift the discrete
    # sweep window from f1r to f2r
    rlen = len(resonances)
    fdata = np.array([])
    ydata = np.array([])
    for ridx, fr in enumerate(resonances):
        try:
            # First sweep from f1 to fr_j - df / 2
            f2r = fr - df / 2
            print(f'[{2*ridx} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
            DM_setup.insert_sweep(f1r, f2r,
                                  count=count1,
                                  save_fields=True,
                                  name=f'Discrete_Sweep_{2*ridx}',
                                  type='Discrete')
            DM_setup.solve()

            # Get data from first sweep
            print(f'Creating even reports ...')
            fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*ridx}')
            # report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='re(Y(LumpPort,LumpPort))')
            # report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='im(Y(LumpPort,LumpPort))')
            report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='re(Y(1, 1))')
            report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='im(Y(1, 1))')
            yre = report_re.get_arrays()
            yim = report_im.get_arrays()
            fdata = np.hstack((fdata, yre[0,:]))
            ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))    

            # Backup data to file
            print(f'Backing up even reports ...')
            with open(f'../data/ydata_{dd}_sweep_{2*ridx}.csv', 'w') as fid:
                fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                                     for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

            # Second sweep from fr_j - df / 2 to fr_j + df / 2
            f1r = f2r + dff
            f2r = fr + df / 2
            print(f'[{2*ridx+1} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
            DM_setup.insert_sweep(f1r, f2r,
                                  count=count2,
                                  save_fields=True,
                                  name=f'Discrete_Sweep_{2*ridx+1}',
                                  type='Discrete')
            DM_setup.solve()

            # Get data from first sweep
            print(f'Creating odd reports ...')
            fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*ridx+1}')
            # report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='re(Y(LumpPort,LumpPort))')
            # report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='im(Y(LumpPort,LumpPort))')
            report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='re(Y(1, 1))')
            report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='im(Y(1, 1))')
            yre = report_re.get_arrays()
            yim = report_im.get_arrays()
            fdata = np.hstack((fdata, yre[0,:]))
            ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))    

            # Backup data to file
            print(f'Backing up even reports ...')
            with open(f'../data/ydata_{dd}_sweep_{2*ridx+1}.csv', 'w') as fid:
                fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                                     for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

            # Next sweep from fr_j + df / 2 to fr_j+1 - df / 2
            f1r = f2r + dff
    
        except Exception as e:
            print(f'Exception on run {ridx}:\n{e}')
            continue
    
    # Last frequency band from fr_j + df / 2 to f2
    f2r = f2
    print(f'[{2*rlen} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
    DM_setup.insert_sweep(f1r, f2r,
                          count=count1,
                          save_fields=True,
                          name=f'Discrete_Sweep_{2*rlen}',
                          type='Discrete')
    DM_setup.solve()
    
    # Get data from first sweep
    print(f'Creating last report ...')
    fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*rlen}')
    # report_re = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='re(Y(LumpPort,LumpPort))')
    # report_im = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='im(Y(LumpPort,LumpPort))')
    report_re = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='re(Y(1, 1))')
    report_im = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='im(Y(1, 1))')
    yre = report_re.get_arrays()
    yim = report_im.get_arrays()
    fdata = np.hstack((fdata, yre[0,:]))
    ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))

    # Backup data to file
    print(f'Backing up even reports ...')
    with open(f'../data/ydata_{dd}_sweep_{2*rlen}.csv', 'w') as fid:
        fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                             for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

## --------------------------------------------------------

## Postprocessing, blackbox quantization

In [ ]:
def get_wp_Cp_Lp(f, y, LJ0=10e-9, CJ0=2e-15, fscale=1e9, use_Zq=False,
                GJ0=7.748e-10):
    """
    Converts frequency-dependent imaginary parts
    of the admittance data to poles,
    capacitances, and inductances
    """
    # Add the junction inductance and capacitance to the
    # total admittance
    # Compute the zero crossings
    fcpy = np.copy(f * fscale)
    ycpy = np.copy(np.complex128(y))
    if LJ0:
        ycpy += (1. / (1j * fcpy * 2 * np.pi * LJ0)) \
          + 1j * fcpy * 2 * np.pi * CJ0
    if LJ0 and GJ0:
        ycpy += (1. / (1j * fcpy * 2 * np.pi * LJ0)) \
             + 1j * fcpy * 2 * np.pi * CJ0 \
             + GJ0
    yim = ycpy.imag
    zero_xing_idxs = np.where(np.diff(np.sign(yim))>0)[0]
    wp = 2 * np.pi * fcpy[zero_xing_idxs]
    yp = yim[zero_xing_idxs]

    # Compute the capacitances from the derivatives
    ypp = np.gradient(ycpy, 2*np.pi*(fcpy[1]-fcpy[0]))
    Cp = 0.5 * np.abs(ypp.imag[zero_xing_idxs])
    Lp = 1. / (wp**2 * Cp)
    Zp = np.sqrt(Lp / Cp)
    Rp = 1. / np.abs(np.real(ycpy[zero_xing_idxs]))
    Qp = wp * Rp * Cp
    
    # Physical constants
    h    = 6.62607015e-34 
    hbar = h / (2 * np.pi)
    e    = 1.602176634e-19
    Phi0 = h / (2 * e)
    
    # Compute Ej, Ec
    if LJ0 and CJ0:
        # Compute the Josephson inductive and charging energies
        Ej = (Phi0 / (2 * np.pi))**2 / LJ0
        Ec = e**2 / (2 * (sum(Cp)+CJ0))

        # Compute inductive energies, EL
        EL = 0.5 * Phi0**2 / Lp

        # Compute the phase zero point fluctuations
        GQ = 2 * e**2 / h
        phip2 = 2 * np.pi * GQ * Zp

        # Compute the participations
        # p = phi2 * 2 * Ej / (hbar * wp)
        pl = Lp / LJ0
        pc = Cp / CJ0

        # Compute the gs / Ds
        Zq = np.sqrt(LJ0 / CJ0)
        gD = np.sqrt(Zp / Zq)

        # Compute the g's in units of Hz, divide by h to get v, Ej = hv
        gpq = (2 * np.pi * GQ * Ej / 24.) * sum(phip2) * np.asarray([[
            np.sqrt(Zpp * Zqq) / h for p, Zpp in enumerate(Zp)]
                               for q, Zqq in enumerate(Zp) ])
        gpq = gpq.reshape([len(gpq), len(gpq)])
        
        # Compute the chis, anharmonicities
        # In units of Hz
        chipp  = -Ec * gD**2 / h
        chippp = np.asarray([-2 * np.sqrt(chipp[p] * chipp[q]) 
                             for p in range(len(chipp))
                             for q in range(len(chipp))])
        chippp = chippp.reshape([len(chipp), len(chipp)])

        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gpq, chippp
    
    else:
        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, None, None, None, None

In [ ]:
def FtoMHz(C):
    """
    Converts Farads to MHz
    """
    MHz  = 1e6
    h    = 6.62607015e-34 
    e    = 1.602176634e-19
    Ec   = e**2 / (2 * C)
    return Ec / h / MHz

In [ ]:
def HtoGHz(L):
    """
    Converts Henrys to GHz
    """
    GHz  = 1e9
    h    = 6.62607015e-34 
    e    = 1.602176634e-19
    Phi0 = h / (2 * e)
    EJ   = (Phi0 / (2 * np.pi))**2 / L
    return EJ / h / GHz

## Load y-data from files if not-overwrite

In [ ]:
def load_ydata_from_sweeps(resonances):
    fdata = np.array([])
    ydata = np.array([], dtype=complex)

    # Iterate over all discrete frequency sweeps
    for i in range(2*len(resonances)+1):
        report_re = fsweep_interp.create_report(f'Discrete_Sweep_{i}', expr='re(Y(LumpPort,LumpPort))')
        report_im = fsweep_interp.create_report(f'Discrete_Sweep_{i}', expr='im(Y(LumpPort,LumpPort))')
        yre = report_re.get_arrays()
        yim = report_im.get_arrays()

        fdata = np.hstack((fdata, yre[0,:]))
        ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))
    
    return fdata, ydata

In [ ]:
def load_ydata_from_txt(filenames, delimiter=','):
    fdata = np.array([])
    ydata = np.array([], dtype=complex)
    # Iterate over all discrete frequency sweeps
    for fname in filenames:
        print(f'Reading {fname} ...')
        data = np.genfromtxt(fname, delimiter=delimiter).T
        f   = data[0]
        yre = data[1]
        yim = data[2]

        fdata = np.hstack((fdata, f))
        ydata = np.hstack((ydata, yre + 1j*yim))
    return fdata, ydata

In [ ]:
if not overwrite:
    res_in = np.zeros(21)
    fdata, ydata = load_ydata_from_files(res_in)

In [ ]:
filenames = [f'../data/ydata_230418_sweep_{i}.csv' for i in range(3)]
#for f in [f'../data/ydata_230326_sweep_{i}.csv' for i in range(3)]:
#    filenames.append(f)
fdata1, ydata1 = load_ydata_from_txt(filenames)

In [ ]:
ydata1

In [ ]:
fdata1

In [ ]:
fdata.size

In [ ]:
ydata.size

## Plot the zero crossings

In [ ]:
# Plot the results, save the figure(s)
fsize = 20
fscale = 1e9
dstr = datetime.datetime.today().strftime('%y%m%d')
wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gD, chippp = \
get_wp_Cp_Lp(fdata1, ydata1, LJ0=100e-9, CJ0=2e-15, fscale=1e9, use_Zq=False)

fig, ax = plt.subplots(1, 1, tight_layout=True)
ax.plot(fdata1, ydata1.imag)
ax.plot(wp/(2*np.pi*fscale), yp.imag, 'o', label='Zero Crossings')
ax.set_ylabel(r'Im $Y(\omega)$', fontsize=fsize)
ax.set_xlabel(r'Frequency [GHz]', fontsize=fsize)
set_axes_fonts(ax, fsize)
set_xaxis_rot(ax, 45.)
#ax.set_xlim([None, 10])
#ax.set_xlim([5.88, 5.95])
ax.set_ylim([-0.05, 0.05])
ax.axhline(y=0, color='k', ls='--')
fig.savefig(f'../figs/zeros_term_bbq_inductive_50um_15mm_{dstr}.pdf', format='pdf')

## Iterate over all coupler offset values, fix LJ=10 nH

In [ ]:
def get_bbq_params(fdata, ydata, NLJ=25, Nmodes=6, LJvals=[0.1e-9, 100e-9]):
    wps = []
    pcs = []
    pls = []
    gDs = []
    Cps = []
    Lps = []
    Qps = []
    chis = []

    # Whether or not to use the largest impedance mode
    # as the junction mode in the calculations
    use_Zq = False
    print_results = True

    LJs = np.logspace(np.log10(LJvals[0]), np.log10(LJvals[1]), NLJ)
    #for idx, x in enumerate(xdata):
    for LJ in LJs:
        wpp, Lp, Cp, Qp, _, _, _, pl, pc, gD, chip \
        = get_wp_Cp_Lp(fdata, ydata, LJ0=LJ, fscale=fscale, use_Zq=use_Zq)
        wps.append(np.sort(wpp[0:Nmodes]))
        #wps.append(wpp[0:Nmodes])
        pls.append(pl[0:Nmodes])
        pcs.append(pc[0:Nmodes])
        gDs.append(gD[0:Nmodes, 0:Nmodes])
        Lps.append(Lp[0:Nmodes])
        Cps.append(Cp[0:Nmodes])
        Qps.append(Qp[0:Nmodes])
        chis.append(chip[0:Nmodes, 0:Nmodes])
        if print_results:
            print(f'wps: {wps}')
        # wps.append(wpp)
        # print(f'LJ: {LJ}, wpp: {wpp}')
    wps = np.asarray(wps).T
    pls  = np.asarray(pls).T
    pcs  = np.asarray(pls).T
    gDs = np.asarray(gDs).T
    Lps = np.asarray(Lps).T
    Cps = np.asarray(Cps).T
    Qps = np.asarray(Qps).T
    chis = np.asarray(chis).T

    return wps, pls, pcs, gDs, Lps, Cps, Qps, chis

In [ ]:
wp0,_ , _, _, _, _, _, _, _, _, _ = get_wp_Cp_Lp(fdata, ydata, fscale=fscale, LJ0=None)

In [ ]:
wps1, pls1, pcs1, gDs1, Lps1, Cps1, Qps1, chis1 = get_bbq_params(fdata, ydata, Nmodes=1, LJvals=[1e-9,100e-9])

In [ ]:
Cps1

In [ ]:
FtoMHz(Cps1[0])

## Plot the $E_C$ of the coupler mode

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NLJ = 25
LJs = np.logspace(np.log10(1e-9), np.log10(100e-9), NLJ)
ax2 = ax.twinx()
for idx, wppp in enumerate(wps1):
    ax.plot(LJs/1e-9, FtoMHz(Cps1[0]),
            'o',
            color=cycle[idx%clen], label=r'$E_C$ [MHz]')
    ax2.plot(LJs/1e-9, HtoGHz(LJs),
           '*', color=cycle[(idx+1)%clen],
            label=r'$E_J$ [GHz]')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$E_C$ [MHz]', fontsize=fsize)
ax2.set_ylabel(r'$E_J$ [GHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
# ax.set_ylim([7.165, 7.2])
set_axes_fonts(ax, fsize)
set_axes_fonts(ax2, fsize)
ax.set_xscale('log')
fig.savefig(f'../figs/modes_term_bbq_inductive_planar_sapphire_{dstr}.pdf',
            transparent=True, format='pdf')

In [ ]:
wps1[2]

In [ ]:
wps1[:, 0]

In [ ]:
wps1[:, -1]

In [ ]:
wps1_sort = np.sort(wps1, axis=0)
wps1_sort

## Plot the modes of coupled structure

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NLJ = 25
LJs = np.logspace(np.log10(1e-9), np.log10(100e-9), NLJ)
for idx, wppp in enumerate(wps1):
    ax.plot(LJs/1e-9, wppp/(2*np.pi*fscale),
            'o', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
    ax.plot(LJs/1e-9, [wp0[idx]/(2*np.pi*fscale)] * len(LJs),
            '--', color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
# ax.set_ylim([7.165, 7.2])
set_axes_fonts(ax, fsize)
leg = set_leg_outside(ax, lsize)
ax.set_xscale('log')
fig.savefig(f'../figs/modes_term_bbq_inductive_planar_sapphire_{dstr}.pdf',
             bbox_extra_artists=(leg, ), bbox_inches='tight',
            transparent=True, format='pdf')

## Plot the splittings of the lowest modes vs LJ

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
ax.plot(LJs/1e-9, (wps1[1]-wps1[0])/(2*np.pi*fscale2),
           'o-',
            color=cycle[0%clen])
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Mode Splitting (1,2) [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
#ax.set_ylim([0, 150])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
fig.savefig(f'../figs/modes_splitting_term_bbq_inductive_planar_sapphire_{dstr}.pdf',
            format='pdf')

## Mode losses

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
for idx, Q in enumerate(Qps1):
    ax.plot(LJs/1e-9, 1./Q,
            'o-', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$Q_p^{-1}$', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_losses_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

## Mode $T_1$'s

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
for idx, Q in enumerate(Qps1):
    ax.plot(LJs/1e-9, 1e6 * Q / wps1[idx],
            'o-', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
ax.plot(LJs/1e-9, 100*np.ones(LJs.size), 'k:',
        lw=2, label=r'100 $\mu$s $T_1$')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$T_1$ [$\mu$s]', fontsize=fsize)
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
#ax.set_ylim([1e-2, 350])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/T1_modes_coupled_{dstr}.pdf',
            format='pdf')

## Black box results

### $\alpha_p$ and $\chi_{pq}$

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
Nmodes = 4
fscale3 = 1e3
for idx in range(Nmodes):
    ax.plot(LJs/1e-9, 0.5 * np.abs(chis1[idx, idx, :]) / fscale3,
            'o-', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|\alpha_p|$ [kHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_alpha_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

In [ ]:
# Chi, lower triangle
chiT = np.array([np.triu(chis1[:,:,i]) for i in range(len(LJs))]).T
gT = np.array([np.triu(gDs1[:,:,i]) for i in range(len(LJs))]).T
NM = 4
chiT[0:NM,0:NM,0]

fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NM = 4
idx = 0
for i in range(NM):
    for j in range(NM):
        if (i != j) and np.any(chiT[i, j, :] != 0.):
            ax.plot(LJs/1e-9, np.abs(chiT[i, j, :]) / fscale3,
                'o-', label=r'$\chi_{%d,%d}$' % (i + 1, j + 1),
                color=cycle[idx%clen])
            idx += 1
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|\chi_{pq}|$ [kHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_chi_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NM = 4
idx = 0
for i in range(NM):
    for j in range(NM):
        if (i != j) and np.any(gT[i, j, :] != 0.):
            ax.plot(LJs/1e-9, np.abs(gT[i, j, :]) / fscale2,
                'o-', label=r'$g_{%d,%d}$' % (i + 1, j + 1),
                color=cycle[idx%clen])
            idx += 1
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|g_{pq}|$ [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_g_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')